# Record Linkage

It refers to the process of identifying and connecting records that correlate with the same entity or individual within one or across multiple data sources. The primary goal of record linkage is to eliminate or reduce duplication and inconsistency in datasets by linking related records together.

This can be particularly useful in various fields and applications, including:

- Health care: Linking patient records from different healthcare providers to create a comprehensive medical history.

- Census and demographics: Matching records from various government agencies to obtain accurate population statistics.

- Customer relationship management: Merging customer data from different sources to provide a unified view of customer interactions.

- Fraud detection: Identifying potentially fraudulent activities by connecting related records.

There are 2 main ways of carrying out Record Linkage :

  - Deterministic Record Linkage - It is a technique that relies on exact matches or predefined rules to link records in a dataset. In this approach, you establish strict criteria for matching records, such as requiring specific fields (e.g., Social Security numbers, unique identifiers, or exact names) to be identical. If all the specified criteria are met, the records are considered a match. It's efficient for datasets where unique identifiers or high-quality matching fields are available. It does not perform well for datasets with a lot of discrepancies or typos.

  - Probabilistic Record Linkage - It is a statistical approach that assesses the likelihood that two records represent the same entity based on the similarity of various attributes or fields. It assigns a probability score to each potential match. A range of matching techniques, such as exact matching, fuzzy matching, and phonetic algorithms, are used to compare the attributes of records. It considers not only the matches but also the mismatches, allowing for variations and errors in the data.Records are linked based on their similarity scores, and a threshold is set to determine which records are considered matches. This threshold can be adjusted to control the trade-off between precision (the accuracy of true matches) and recall (the ability to capture all true matches).

Note , whether to use a Deterministic approach or Probabilistic one depends on  the nature of the data, the quality of the matching fields, and the desired balance between precision and recall in the linkage process.

RecordLinkage Steps using

**Task**

We have two datasets: ***Credit Data and Customer Data***. The objective of this task is to determine which records in the Credit Data match the corresponding customer information in the Customer Data. This matching process can be utilized, for example, by a bank to assess whether a customer is eligible for a loan, taking into account their credit-related information.

Note , This is Synthetic data that was generated using the Faker Library for the purpose of this task.

**Preprocessing.**

There are 2 functions from recordlinkage library that can assist in preprocessing :

- recordlinkage.preprocessing.clean() -  Cleans strings by removing undesired tokens, whitespace, and brackets, making the data more consistent and suitable for linkage or comparison tasks.
        
- recordlinkage.preprocessing.phonetic() - Converts strings into their phonetic codes. Phonetic algorithms, like Soundex, index words based on their pronunciation. Usecase : Useful when dealing with names that might have multiple spellings but sound similar. Converting strings into phonetic codes can help in identifying similar-sounding words, even if they are spelled differently.

Note, you don't have to use both of them. Again, it depends on what you want to achieve and the nature of your data.

In [ ]:
!pip install recordlinkage


In [ ]:
# Load Data
import recordlinkage as rl
import pandas as pd
import numpy as np

# Specify your paths

df_credit = pd.read_csv('./credit_data.csv', index_col='Customer ID')
df_customer = pd.read_csv('./customer_data.csv', index_col='Customer ID')



In [ ]:
#
df_credit.head()

,Full Name,Credit Score,Credit Limit,Outstanding Debt,SSN
Customer ID,,,,,
201,Michael Hunter,698,"$11,890","$1,165",681516294
202,Kevin Hanna,666,"$13,376","$2,990",427278963
203,Amy King,703,"$9,969","$4,964",125521435
204,Elizabeth Hunter,722,"$10,866","$3,389",890269661
205,Laura Pierce,655,"$13,268","$1,570",440150217


In [ ]:
df_customer.head()

,Customer Name,Address,Phone Number,SSN
Customer ID,,,,
101,Michael Hunterr,"9115 Hendricks Wall\nNew Meganside, MP 68938",661.923.4168x02753,681516394
102,Kevin,"PSC 1969, Box 7516\nAPO AE 75166",+1-893-342-9859x743,427378963
103,Emmy Kinng,"415 Washington Curve\nLake Cheryl, PA 42125",760-971-0604x225,325521435
104,Ms. Elizabeth Huntar,914 Stephen Road Suite 464\nNorth Mariahcheste...,(401)605-4912,840269661
105,Laura Pierce,00207 Joel Landing Suite 636\nSouth Kimberlyvi...,001-965-395-2767x35364,440150217


In [ ]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 101 to 1100
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Customer Name   1000 non-null   object
 1   Address         1000 non-null   object
 2   Phone Number    1000 non-null   object
 3   SSN             1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 39.1+ KB


In [ ]:
df_credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 201 to 1200
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Full Name         1000 non-null   object
 1   Credit Score      1000 non-null   int64 
 2   Credit Limit      1000 non-null   object
 3   Outstanding Debt  1000 non-null   object
 4   SSN               1000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 46.9+ KB


In [ ]:
# Clean String Data i.e  removing undesired tokens, whitespace, and so on
# We will pass in individual columns as the function takes in a Series Object

from recordlinkage.preprocessing import clean

# Customer Data
df_customer['Customer Name '] = clean(df_customer['Customer Name '])
df_customer['Address'] = clean(df_customer['Address'])

# Credit Data
df_credit['Full Name'] = clean(df_credit['Full Name'])





In [ ]:
# After cleaning
df_customer.head()

,Customer Name,Address,Phone Number,SSN
Customer ID,,,,
101,michael hunterr,9115 hendricks wallnew meganside mp 68938,661.923.4168x02753,681516394
102,kevin,psc 1969 box 7516apo ae 75166,+1-893-342-9859x743,427378963
103,emmy kinng,415 washington curvelake cheryl pa 42125,760-971-0604x225,325521435
104,ms elizabeth huntar,914 stephen road suite 464north mariahchester ...,(401)605-4912,840269661
105,laura pierce,00207 joel landing suite 636south kimberlyview...,001-965-395-2767x35364,440150217


In [ ]:
# After Cleaning
df_credit.head()

,Full Name,Credit Score,Credit Limit,Outstanding Debt,SSN
Customer ID,,,,,
201,michael hunter,698,"$11,890","$1,165",681516294
202,kevin hanna,666,"$13,376","$2,990",427278963
203,amy king,703,"$9,969","$4,964",125521435
204,elizabeth hunter,722,"$10,866","$3,389",890269661
205,laura pierce,655,"$13,268","$1,570",440150217


**Indexing**

This involves creating possible matching records within our data. These matching pairs are called *candidate links or candidate matches*. To do so, various Indexing Algorithms are used. They include :
- Full Indexing - Also known as *full cartesian product indexing*, it creates candidate links by forming all possible unique combinations between records in one dataset and those in another dataset. Essentially, every record in the first dataset is compared to every record in the second dataset. Use this method when the datasets are small in size. It is quite computationally expensive.

- Block Indexing - Candidate links are created by partitioning records into distinct groups or "blocks", based on one or more key attributes. Only records within the same block are compared, thus significantly reducing the number of comparisons required. The key attribute(s) used for blocking should have a high probability of being the same for true matches.

Other methods include SortedNeighbourhood and Random Indexing. Please refer to the documentation on this.

In [ ]:
# We will do Block Indexing as we have a column matching across the datasets - SSN.
# By the time I was doing this, I did not have adequate resources.
# Best method to use is Full Indexing given the size of our datasets

import recordlinkage as rl

indexer = rl.Index()

# Cnadidate Block - use common column across the datasets
indexer.block(left_on='SSN',right_on='SSN')

candidate_links = indexer.index(df_customer,df_credit)

print(f"Total number of candidate links - {len(list(candidate_links))}")

Total number of candidate links - 977


**Comparing**

Comparing involves assessing pairs of records to determine how similar they are across various attributes or fields. Once *potential* candidate pairs are identified, these pairs undergo a comparison to produce a similarity score. This score is then used to decide if the pair is a true match or not.

The comparison can involve:

- Exact Matching - Check if the values in the fields are exactly the same. Use it when you have limited set of unique values within the columns. The data should also be of high quality.

- Fuzzy Matching - Using algorithms to determine the similarity between values. String Compare uses algorithms such as jarowinkler, cosine, jaro, and levenshtein.

**Jarowinkler** is used to strings that match from the beginning e.g Names of People.

**Levenshtein** Calculates the minimum number of single-character edits (i.e., insertions, deletions, or substitutions) to change one word into the other. Used when there are typos or when dealing with Addresses.

- Numeric or Date Comparisons - For numeric or date fields, differences can be computed directly or categorizsd into bands (e.g., age difference of less than 5 years, between 5 and 10 years, etc.).

- Custom Comparisons: Depending on domain-specific knowledge, custom comparison functions can be employed. For example, in a medical context, two different codes might represent the same condition, and a custom function can identify them as a match.

Note, Comparing is a crucial step in record linkage as it directly influences the quality of the linkage.

In [ ]:
import recordlinkage as rl

compare = rl.Compare()
compare.string(left_on='Customer Name ',right_on='Full Name',label='Customer Full Name',method='jarowinkler')

features = compare.compute(candidate_links, df_customer, df_credit)

**Matches**



To get the strongest matches, we set the threshold slightly above mean similarity for all features.

If we use a number bigger than 1, then we have 0 links.
If the threshold value is smaller, we have more weaker links.

Normally, I first establish the mean similarity score for all features and from there determine the threshold value.

In [ ]:
# Mean similarity scores for all the features,
features.mean().sum()

0.9960372274093156

In [ ]:

matches = features[features.sum(axis=1) >0.9]
print('number of record matches:',len(matches))

number of record matches: 966


In [ ]:
# Customer ID_1 represents Customer IDs from df_customer data
# Customer ID_2 represents Customer IDs from df_credit data
# Customer Full Name is the Similarity Score
# You can proceed to match up the various IDs with the original datasets to obtain the result dataset

matches

,,Customer Full Name
Customer ID_1,Customer ID_2,
105,205,1.000000
106,206,1.000000
110,210,0.983333
111,211,1.000000
113,213,1.000000
...,...,...
1093,1193,0.985714
1094,1194,1.000000
1095,1195,1.000000
